In [29]:
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
from transformers import pipeline

from __future__ import unicode_literals
import spacy,en_core_web_sm
from spacy.lang.en import English
from spacy.matcher import Matcher
import textacy
import string
from wordcloud import WordCloud, STOPWORDS 
import numpy as np
import nltk
import locationtagger
from difflib import SequenceMatcher
import pickle
import gensim
from gensim.models import Word2Vec
from gensim import corpora


In [ ]:
class MlSkillsOne:
    
    def __init__(self):
        pass
        self.all_records_raw_df = self.read_data_individual_topics()
        self.all_records_cleaned_df = self.data_cleaner()
        self.target_phrase = "Jeu de Paume is an excellent art gallery in Paris"        

    def read_data_individual_topics(self):

        path = 'individualTopics_27-01-22/'
        all_records = []

        for fname in glob.glob(path + '*.pickle'):
            obj = pd.read_pickle(fname)
            record = [obj['id'],obj['name'],obj['audience_size'],
                      obj['country'],obj['topic']]
            all_records = all_records + [record]
        
        all_records_df = pd.DataFrame.from_records(all_records)
        all_records_df.columns = ['id','name','audience_size','country','topic']
        
        return all_records_df
    
    def number_of_verb(self, string):
        verbs = []
        pattern = [{'POS': 'VERB', 'OP': '?'},\
               {'POS': 'VERB', 'OP': '+'}]
        doc = textacy.make_spacy_doc(string, lang='en_core_web_sm')
        lists = textacy.extract.matches.token_matches(doc, [pattern])
        for list in lists:
            verbs.append(list.text)
            
        return len(verbs)
    
    def number_letters(self, string):
        return len([i for i in string if i.isalpha()])
    
    def location(self, string):
        place_entity = locationtagger.find_locations(text = string)
        countries = place_entity.countries
        regions = place_entity.regions
        cities = place_entity.cities
        X = countries + regions + cities
        return X

    def similarity(self, row):
        return SequenceMatcher(None, row, "Jeu de Paume is an excellent art gallery in Paris").ratio()
    
    def data_cleaner(self):

        self.all_records_raw_df["name_cleaned"] = self.all_records_raw_df.name\
            .apply(lambda row: row.translate(str.maketrans('', '', string.punctuation)))
        """
        self.all_records_raw_df["number_of_verb"] = self.all_records_raw_df.name_cleaned\
            .apply(lambda row: self.number_of_verb(row))
        
        self.all_records_raw_df["number_words"] = self.all_records_raw_df.name_cleaned\
            .apply(lambda row: len(row.split()))
            
        self.all_records_raw_df["number_letters"] = self.all_records_raw_df.name_cleaned\
            .apply(lambda row: self.number_letters(row))
        
        self.all_records_raw_df["number_letters_words_verbs"] = self.all_records_raw_df.number_letters.\
            apply(lambda row: [row]) + self.all_records_raw_df.number_words.apply(lambda row: [row]) + \
            self.all_records_raw_df.number_of_verb.apply(lambda row: [row])
         
        self.all_records_raw_df["name_entity"] =  self.all_records_raw_df.name_cleaned\
            .apply(lambda row: self.location(row))  
        """ 
        self.all_records_raw_df["similarity"] =  self.all_records_raw_df.name_cleaned\
            .apply(lambda row: self.similarity(row))
        
        self.all_records_raw_df["target_phrase"] = "Jeu de Paume is an excellent art gallery in Paris"
        
        return self.all_records_raw_df
        
    def word_cloud(self):
        string = self.all_records_raw_df.name_cleaned
        comment_words = ''
        stopwords = set(STOPWORDS)
        for val in string:

            # typecaste each val to string
            val = str(val)

            # split the value
            tokens = val.split()

            # Converts each token into lowercase
            for i in range(len(tokens)):
                tokens[i] = tokens[i].lower()

            comment_words += " ".join(tokens)+" "

        wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10).generate(comment_words)

        # plot the WordCloud image                      
        plt.figure(figsize = (8, 8), facecolor = None)
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.tight_layout(pad = 0)

        plt.show()   

In [2]:
class MlSkillsTwo:
    
    def __init__(self):
        self.instance_data = self.read_instances()
        self.variables_df = pd.DataFrame(self.read_variables(),\
                                      columns= ('var1','var2','var3','var4','var5'))
        self.instances_list = self.transform_instance()
        self.flat_instace = self.concatenation()
        
    def read_instances(self):
        
        with open("data.pickle", "rb") as file:
            instances = pickle.load(file)
            
        return instances    
    
    def read_variables(self):
    
        with open("variables.pickle", "rb") as file:
            variables = pickle.load(file)
            
        return variables
    
    def transform_instance(self):
            elements = []
            instances_list = []
            total_df = pd.DataFrame()
            for instance in range(0, len(self.instance_data[:])):

                elements = []

                for ele in range(len(self.instance_data[instance][0])):
                    elements.append("ele" + str(ele))

                #print(self.instance_data[instance][0])
                #print(elements)
                total_df = pd.DataFrame(self.instance_data[instance], columns=elements)
                instances_list.append(total_df)
                #total_df["var1"] = self.variables[instance]

            return instances_list
    
    
    def concatenation(self):
        instance = []
        for instances_df in range(0, len(self.instances_list)):
            for column in self.instances_list[instances_df].columns:
                instance.append(pd.concat([self.transform_instance()[instances_df]\
                                [column],self.variables_df], axis=1).\
                                rename(columns = {column:'ele', 'var1':'var1',\
                                'var2':'var2', 'var3':'var3', 'var4':'var4', 'var5':'var5'}))
                
        instance = pd.concat(instance)
        return  instance



In [3]:
to_trantor = MlSkillsTwo()

In [4]:
to_trantor.flat_instace

,ele,var1,var2,var3,var4,var5
0,nan,30.0,55.0,97.0,85.0,50.0
1,nan,2.0,60.0,90.0,85.0,95.0
2,nan,65.0,96.0,90.0,65.0,90.0
3,nan,85.0,45.0,5.0,30.0,10.0
4,Mercadillo,90.0,30.0,65.0,20.0,5.0
...,...,...,...,...,...,...
997,nan,98.0,25.0,10.0,10.0,2.0
998,nan,99.0,4.0,45.0,5.0,1.0
999,nan,80.0,20.0,50.0,15.0,15.0
1000,nan,99.0,10.0,15.0,5.0,1.0


In [7]:
df_clean = to_trantor.flat_instace.replace('nan', np.nan).dropna()

## Breve investigacion de WE

In [31]:
from collections import defaultdict

In [40]:
df_clean['ele'][8][18:19].tolist()

['La ruleta de la suerte']

In [33]:
#creating a dictionary

docs  = df_clean['ele'][8][18:19].tolist()
stoplist = set('for a of the and to in'.split())
txts = [[word for word in document.lower().split() if word not in stoplist]for document in docs]

In [34]:
frequency = defaultdict(int)

for text in txts:

     for token in text:

        frequency[token] += 1


In [37]:
txts = [[token for token in text if frequency[token] > 1]for text in txts]
gensim_dictionary = corpora.Dictionary(txts)
print(gensim_dictionary.token2id)
doc_new = "corpus consists document"
vec = gensim_dictionary.doc2bow(doc_new.lower().split())
print("\n",vec)

{'la': 0}

 []


In [38]:
gensim_corpus = [gensim_dictionary.doc2bow(text) for text in txts]
print(gensim_corpus)

[[(0, 2)]]
